In [197]:
#general imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.model_selection import KFold

In [198]:
#Read dataset and make ydata-profiling dashboard

#from ydata_profiling import ProfileReport

df = pd.read_csv("../data/datasetRuntimeData.csv")
#Drop index col
df = df.drop(["Unnamed: 0"], axis=1)
#Remove special symbole
df['AutoML_adapter'] = df['AutoML_adapter'].str.replace(':', '')
#Apply one hot encoding
df = pd.get_dummies(df, columns=['AutoML_adapter'], prefix='', prefix_sep='')

#profile = ProfileReport(df, title="Profiling Report")
#profile.to_notebook_iframe()

In [199]:
df

,dataset_size_in_mb,dataset_rows,dataset_cols,missing_values,duplicated_rows,duplicated_cols,outliers,runtime_limit,autogluon,evalml,flaml,gama,h2o_automl,lama,pycaret,tpot
0,18.085659,539383,8,0,154622,0,99,40,False,False,False,False,True,False,False,False
1,18.085659,539383,8,0,154622,0,99,5,False,True,False,False,False,False,False,False
2,18.085659,539383,8,0,154622,0,99,640,False,False,False,True,False,False,False,False
3,18.085659,539383,8,0,154622,0,99,10,True,False,False,False,False,False,False,False
4,18.085659,539383,8,0,154622,0,99,20,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,0.197092,683,36,0,41,0,0,20,False,False,False,False,True,False,False,False
316,0.197092,683,36,0,41,0,0,5,True,False,False,False,False,False,False,False
317,0.197092,683,36,0,41,0,0,5,False,False,False,False,False,False,False,True
318,0.197092,683,36,0,41,0,0,20,False,False,True,False,False,False,False,False


In [200]:
#Split data all AutoML together

X = df.drop(["runtime_limit"], axis=1)
y = df["runtime_limit"]

from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X = pd.DataFrame(x_scaler.fit_transform(X), columns=X.columns)
y = pd.Series(y_scaler.fit_transform(y.values.reshape(-1, 1)).flatten())


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [201]:
#Create prediction plot
def create_prediction_plot(y_test, predictions):
    prediction_results = pd.DataFrame({
        'runtime_limit_is': y_test,
        'runtime_limit_predicted': predictions
    })

    best_case_x = [0, 5, 10, 20, 40, 80, 160, 320, 640]
    best_case_y = [0, 5, 10, 20, 40, 80, 160, 320, 640]
    plt.figure(figsize=(10, 6))
    sns.scatterplot(
        x='runtime_limit_predicted', 
        y='runtime_limit_is', 
        data=prediction_results, 
        color='gray', marker='o'  # Using a distinct color palette
    )

    plt.plot(best_case_x, best_case_y)
    plt.xscale('log', base=10)  # Logarithmic scale for x-axis
    plt.yscale('log', base=10)  # Logarithmic scale for y-axis


    # Find the limits in log space
    x_min, x_max = 1, 100
    y_min, y_max = 1, np.exp(6.6)

    # Determine the limits to make them symmetrical in log space
    log_min = min(np.log10(x_min), np.log10(y_min))
    log_max = max(np.log10(x_max), np.log10(y_max))

    # Apply the symmetrical limits
    plt.xlim([10**log_min, 10**log_max])
    plt.ylim([10**log_min, 10**log_max])




    plt.xlabel('Optimal runtime predicted')
    plt.ylabel('Optimal runtime measured')
    #plt.legend(title='AutoML Solution', bbox_to_anchor=(1.05, 0.5), loc='center left')
    #plt.title('Actual vs Predicted Runtime Limits')
    plt.grid(True)
    #plt.legend(title='Series')
    plt.show()
    

In [202]:
X

,dataset_size_in_mb,dataset_rows,dataset_cols,missing_values,duplicated_rows,duplicated_cols,outliers,autogluon,evalml,flaml,gama,h2o_automl,lama,pycaret,tpot
0,0.034972,0.928289,0.000932,0.0,1.000000,0.0,0.000234,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.034972,0.928289,0.000932,0.0,1.000000,0.0,0.000234,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.034972,0.928289,0.000932,0.0,1.000000,0.0,0.000234,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.034972,0.928289,0.000932,0.0,1.000000,0.0,0.000234,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.034972,0.928289,0.000932,0.0,1.000000,0.0,0.000234,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,0.000352,0.000315,0.007454,0.0,0.000265,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
316,0.000352,0.000315,0.007454,0.0,0.000265,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
317,0.000352,0.000315,0.007454,0.0,0.000265,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
318,0.000352,0.000315,0.007454,0.0,0.000265,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [203]:

selected_feature = ["autogluon", "evalml", "flaml", "gama", "h2o_automl", "lama", "pycaret", "tpot", "dataset_rows"]

In [204]:


def train_model(model, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mae_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train[selected_feature], y_train)
        predictions = model.predict(X_test[selected_feature])
        
        y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
        predictions = pd.Series(y_scaler.inverse_transform(predictions.reshape(-1, 1)).flatten())
        
        # Compute MAE
        mae = MAE(predictions, y_test_)
        mae_scores.append(mae)
        
    avg_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)  # Compute standard deviation
    
    print("###############################################")
    print("MAE Scores:", mae_scores)
    print(f"{type(model)} Average MAE across {n_splits} folds: {round(avg_mae, 4)} ± {round(std_mae, 4)}")
    
    
    return model

In [205]:
# def train_model(model,):
#     model.fit(X_train[selected_feature], y_train)
#     predictions = pd.Series(model.predict(X_test[selected_feature]))

#     y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
#     predictions = pd.Series(y_scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten())

#     # Compute MAE
#     mae = MAE(predictions, y_test_)
    
#     print(f"{type(model)} Mean Absolute Error (MAE):", round(mae))
#     return model

In [206]:
X

,dataset_size_in_mb,dataset_rows,dataset_cols,missing_values,duplicated_rows,duplicated_cols,outliers,autogluon,evalml,flaml,gama,h2o_automl,lama,pycaret,tpot
0,0.034972,0.928289,0.000932,0.0,1.000000,0.0,0.000234,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.034972,0.928289,0.000932,0.0,1.000000,0.0,0.000234,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.034972,0.928289,0.000932,0.0,1.000000,0.0,0.000234,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.034972,0.928289,0.000932,0.0,1.000000,0.0,0.000234,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.034972,0.928289,0.000932,0.0,1.000000,0.0,0.000234,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,0.000352,0.000315,0.007454,0.0,0.000265,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
316,0.000352,0.000315,0.007454,0.0,0.000265,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
317,0.000352,0.000315,0.007454,0.0,0.000265,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
318,0.000352,0.000315,0.007454,0.0,0.000265,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [207]:
def train_keras(n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mae_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Define the model
        model = Sequential([
            Dense(len(X_train[selected_feature].columns), activation='relu', input_shape=(len(X_train[selected_feature].columns),)),
            Dense(1, activation='linear')
        ])
        
        # Define ModelCheckpoint callback
        checkpoint_callback = ModelCheckpoint(
            './keras_models/2_3_1_best_model.keras', 
            monitor='val_mae',  # Monitor validation loss
            save_best_only=True,  # Save only when validation loss improves
            mode='auto'
        )
        
        early_stopping_callback = EarlyStopping(
            monitor='val_mae',  # Monitor validation MAE
            patience=25,  # Stop training if the validation MAE doesn't improve for 10 epochs
            restore_best_weights=True  # Restore the model weights from the best epoch
        )
        
        # Compile the model
        model.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError(), metrics=['mae'])
        model.fit(X_train[selected_feature], y_train, epochs=500, batch_size=16, verbose=0, callbacks=[checkpoint_callback, early_stopping_callback], validation_data=(X_test[selected_feature], y_test))
        
        model = tf.keras.models.load_model('./keras_models/2_3_1_best_model.keras')
        predictions = pd.Series(model.predict(X_test[selected_feature]).flatten())
        
        y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
        predictions = pd.Series(y_scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten())
        
        # Compute MAE
        mae = MAE(predictions, y_test_)
        mae_scores.append(mae)
    
    avg_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)  # Compute standard deviation
    
    print("###############################################")
    print("MAE Scores:", mae_scores)
    print(f"{type(model)} Average MAE across {n_splits} folds: {round(avg_mae, 4)} ± {round(std_mae, 4)}")
    
    
    return model


In [208]:
# def train_keras():
#     import tensorflow as tf
#     from tensorflow import keras
#     from tensorflow.keras.models import Sequential
#     from tensorflow.keras.layers import Dense, Convolution1D
#     from tensorflow.keras.callbacks import ModelCheckpoint

#     # Define the model
#     model = Sequential([
#         Dense(len(X_train.columns), activation='relu', input_shape=(len(X_train.columns),)),  # Input layer (10 features)
#         #Dense(12, activation='relu'),  # Hidden layer 1
#         #Dense(6, activation='relu'),  # Hidden layer 2
#         Dense(1, activation='linear')  # Output layer (for binary classification)
#     ])

#     # Define ModelCheckpoint callback
#     checkpoint_callback = ModelCheckpoint(
#         './keras_models/2_3_1_best_model.h5', 
#         monitor='val_mae',  # Monitor validation loss
#         save_best_only=True,  # Save only when validation loss improves
#         mode='auto',  # We want to minimize validation loss
#         #verbose=1
#     )

#     # Compile the model
#     model.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError(), metrics=['mae'])
#     model.fit(X_train, y_train, epochs=500, batch_size=16, verbose=0, callbacks=[checkpoint_callback], validation_data=(X_test, y_test))
    
#     model = tf.keras.models.load_model('./keras_models/2_3_1_best_model.h5')
#     predictions = pd.Series(model.predict(X_test).flatten())


#     y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
#     predictions = pd.Series(y_scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten())


#     # Compute MAE
#     mae = MAE(predictions, y_test_)

#     print(f"{type(model)} Mean Absolute Error (MAE):", round(mae))
#     return model

In [209]:
def train_lgbm_model(n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mae_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        train_data = lgb.Dataset(X_train[selected_feature], label=y_train)
        test_data = lgb.Dataset(X_test[selected_feature], label=y_test, reference=train_data)
        
        params = {
            'objective': 'regression',
            'metric': 'mae',
            'verbose': -1
        }
        
        num_round = 100
        model = lgb.train(params, train_data, num_round, valid_sets=[test_data])
        
        predictions = pd.Series(model.predict(X_test[selected_feature], num_iteration=model.best_iteration))
        
        y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
        predictions = pd.Series(y_scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten())
        
        mae = MAE(predictions, y_test_)
        mae_scores.append(mae)
    
    avg_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)  # Compute standard deviation
    
    print("###############################################")
    print("MAE Scores:", mae_scores)
    print(f"{type(model)} Average MAE across {n_splits} folds: {round(avg_mae, 4)} ± {round(std_mae, 4)}")
    
    
    return model


In [210]:

# def train_lgbm_model():
#     train_data = lgb.Dataset(X_train[selected_feature], label=y_train)
#     test_data = lgb.Dataset(X_test[selected_feature], label=y_test, reference=train_data)



#     # Define parameters for the LightGBM model
#     params = {
#         'objective': 'regression',  # Set the objective as regression
#         'metric': 'mae',            # Use mean absolute error as the evaluation metric
#         'verbose': 1                # Disable verbose output
#     }

#     # Train the LightGBM model
#     num_round = 100
#     model = lgb.train(params, train_data, num_round)

#     # Make predictions on the test set
#     predictions = pd.Series(model.predict(X_test[selected_feature], num_iteration=model.best_iteration))
    

#     y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
#     predictions = pd.Series(y_scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten())

#     # Compute MAE
#     mae = MAE(predictions, y_test_)

#     print(f"{type(model)} Mean Absolute Error (MAE):", round(mae))
#     return model

In [211]:

models = { "Baseline": DummyRegressor(strategy="median"), 
          "LightGBM": None, 
          "Linear Regression": LinearRegression(), 
          "Decision Tree": DecisionTreeRegressor(random_state=42), 
          "Sklearn Neural Network": MLPRegressor(random_state=42), 
          "Keras Neural Network": None, 
          "Ridge": Ridge(), 
          "Lasso": Lasso(), 
          "Elastic": ElasticNet(), 
          "Random Forest": RandomForestRegressor(), 
          "Bayesian": BayesianRidge(), 
          "SVM": SVR()}


for model_name, model in models.items():
    if model_name == "LightGBM":
        models[model_name] = train_lgbm_model()
    elif model_name == "Keras Neural Network":
        models[model_name] = train_keras()
    else:
        models[model_name] = train_model(model)


###############################################
MAE Scores: [104.375, 105.15625, 155.15625, 121.5625, 106.40625, 93.4375, 51.71875, 85.3125, 75.46875, 109.84375]
<class 'sklearn.dummy.DummyRegressor'> Average MAE across 10 folds: 100.8438 ± 26.2209
###############################################
MAE Scores: [107.19115125514385, 104.33856624719758, 150.03663807997742, 151.11812912430082, 119.19817438657964, 94.51902199361629, 81.09641564090123, 117.12305851830476, 122.45232952894881, 122.77818204254035]
<class 'lightgbm.basic.Booster'> Average MAE across 10 folds: 116.9852 ± 20.9151
###############################################
MAE Scores: [108.3770751953125, 114.6044921875, 157.2705078125, 156.083984375, 118.8409423828125, 99.375, 84.04541015625, 113.1817626953125, 116.26861572265625, 124.490966796875]
<class 'sklearn.linear_model._base.LinearRegression'> Average MAE across 10 folds: 119.2539 ± 21.5635
###############################################
MAE Scores: [153.75, 120.15625, 16

c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
###############################################
MAE Scores: [95.06317062024027, 96.6105987727642, 142.79330112040043, 118.35079504549503, 111.0671778023243, 89.35367432236671, 50.343459598720074, 89.78273078799248, 78.00881791114807, 108.90124087780714]
<class 'keras.src.models.sequential.Sequential'> Average MAE across 10 folds: 98.0275 ± 23.524
###############################################
MAE Scores: [108.81165978580005, 109.8392276011684, 159.01225999672153, 158.37731826205746, 117.70599686278429, 100.1547789841133, 84.42804978684845, 110.06551489097146, 116.73168579873328, 122.92995649669697]
<class 'sklearn.linear_model._ridge.Ridge'> Average MAE across 10 folds: 118.8056 ± 22.3413
###############################################
MAE Scores: [118.50260416666667, 144.55729166666669, 182.861328125, 160.78125, 130.59895833333334, 124.61805555555556, 99.990234375, 112.53472222222223, 125.771484375, 132.48046875]
<class 'sklearn.linear_model._coo

In [212]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.lines import Line2D

# Combine train and test data for predictions
# X_all = pd.concat([X_train[selected_feature], X_test[selected_feature]])
# y_all = pd.concat([y_train, y_test])
X_all = X
y_all = y


# Make predictions
predictions = pd.Series(models["Keras Neural Network"].predict(X_all))

y_all = pd.Series(y_scaler.inverse_transform(y_all.values.reshape(-1, 1)).flatten())
predictions = pd.Series(y_scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten())

# Create a DataFrame for results
result_df = X_all.copy()
result_df['runtime_limit_is'] = y_all.values
result_df['runtime_limit_predicted'] = predictions

# Assign categories manually based on the one-hot encoded columns
def get_category(row, category_columns):
    for col in category_columns:
        if row[col] == 1:
            return col.replace('category_', '')
    return None

# Find the one-hot encoded column where it's true for each row
category_columns = ["autogluon", "evalml", "flaml", "gama", "h2o_automl", "lama", "pycaret", "tpot"]
result_df['category'] = result_df.apply(get_category, axis=1, category_columns=category_columns)

# Count the number of dots per x-axis class
dot_counts = result_df['runtime_limit_is'].value_counts().sort_index()

# Plot using Seaborn
plt.figure(figsize=(12, 8))
scatter = sns.scatterplot(
    y='runtime_limit_predicted',
    x='runtime_limit_is',
    hue='category',
    data=result_df,
    palette=sns.color_palette("hsv", len(result_df['category'].unique()))  # Using a distinct color palette
)

# Add a custom legend for the categories
handles, labels = scatter.get_legend_handles_labels()
category_legend = plt.legend(handles, labels, title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.gca().add_artist(category_legend)

# Create a second legend for the counts with no markers
count_labels = [f'{int(key)}: {value}' for key, value in dot_counts.items()]
count_legend_handles = [Line2D([0], [0], color='white', label=label) for label in count_labels]
count_legend = plt.legend(count_legend_handles, count_labels, title='Runtime limit: Counts', bbox_to_anchor=(1.05, 0.5), loc='center left')

# Add the best case line
best_case_x = [0, 5, 10, 20, 40, 80, 160, 320, 640]
best_case_y = [0, 5, 10, 20, 40, 80, 160, 320, 640]
plt.plot(best_case_x, best_case_y)

# Set logarithmic scales
plt.xscale('log', base=10)
plt.yscale('log', base=10)

# Set labels and title
plt.ylabel('Optimal runtime limit predicted')
plt.xlabel('Optimal runtime limit measured')
plt.title('Optimal runtime limit prediction by AutoML')

# Ensure grid is enabled
plt.grid(True)

# Adjust layout to make space for legends
plt.subplots_adjust(right=0.75)

plt.show()


InvalidArgumentError: Graph execution error:

Detected at node sequential_62_1/dense_124_1/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Program Files\WindowsApps\Py